In [18]:
# IMPORTS
import random
import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

from functools import partial
from ProblemScripts import load_problem_KP

import os
import pickle
import json

### Fitness functions

In [2]:
def OneMax_fitness(individual):
    """ Function calculates fitness for OneMax problem individual """
    fitness = sum(individual)
    return (fitness,)

def eval_ind_kp(individual, items_dict, capacity, penalty=1):
    """ Function calculates fitness for knapsack problem individual """
    n_items = len(individual)
    weight = sum(items_dict[i][1] * individual[i] for i in range(n_items)) # Calc solution weight
    value = sum(items_dict[i][0] * individual[i] for i in range(n_items)) # Calc solution value
    
    # Check if over capacity and return reduced value
    if weight > capacity:
        if penalty == 1:
            value_with_penalty = value - (weight - capacity)
            return (value_with_penalty,)
        else:
            return (0,)
    return (value,) # Not over capacity return value

In [3]:
def generate_zero_solution(length):
    return np.zeros(length, dtype=int)

### Algorithms

In [4]:
def EA(NGEN, popsize, tournsize, MUTPB, indpb, len_sol, fitness_function, fitness_params=None, starting_solution=None, true_fitness_function=None, true_fitness_params=None, n_elite=1):
    # Check if the fitness and individual creators have been defined; if not, define them
    if not hasattr(creator, "OneMax_fitness"):
        creator.create("OneMax_fitness", base.Fitness, weights=(1.0,))
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.OneMax_fitness)

    # Define the toolbox used for the evolutionary algorithm
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)  # Define an attribute as a random binary value
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len_sol)  # Create an individual with len_sol attributes
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # Create a population of individuals
    
    # Register the evaluation function, allowing additional parameters to be passed
    if fitness_params is None:
        fitness_params = {}
    toolbox.register("evaluate", lambda ind: fitness_function(ind, **fitness_params))
    
    toolbox.register("mutate", tools.mutFlipBit, indpb=indpb)  # Mutation function: flip a bit with a given probability
    toolbox.register("select", tools.selTournament, tournsize=tournsize)  # Selection function: tournament selection

    # Create an initial population of individuals
    population = toolbox.population(n=popsize)

    # If a starting solution is provided, set all individuals to that solution
    if starting_solution is not None:
        for ind in population:
            ind[:] = starting_solution[:]

    # Evaluate the initial population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    # Initialise data to record every generation's population, best solutions, best fitness values, and true fitness values
    all_generations, best_solutions, best_fitnesses, true_fitnesses = ([] for _ in range(4))
    data = [population, all_generations, best_solutions, best_fitnesses, true_fitnesses]
    # Record initial population
    record_population_state(data, toolbox, true_fitness_function, true_fitness_params)

    # Evolutionary loop for each generation
    for gen in range(NGEN):
        # Select the offspring using tournament selection (cloning to avoid modifying the original individuals)
        n_offspring = popsize - n_elite
        offspring = [toolbox.clone(toolbox.select(population, 1)[0]) for _ in range(n_offspring)]

        # Apply mutation on the offspring with probability MUTPB
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)  # Mutate the individual
                del mutant.fitness.values  # Delete fitness to mark it as needing reevaluation

        # Evaluate the individuals with an invalid fitness (those that were mutated)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Select the best individuals from the current population to keep (elitism)
        elites = tools.selBest(population, n_elite)

        # Replace the weakest individuals in the population with the offspring
        for mutant in offspring:
            weakest_idx = min(range(len(population)), key=lambda idx: population[idx].fitness.values)  # Find the weakest individual
            population[weakest_idx] = mutant  # Replace it with the mutated individual

        # Add the elite individuals back to the population
        population.extend(elites)

        # Create the new population by combining elites and offspring
        # population = elites + offspring

        # Record current population
        record_population_state(data, toolbox, true_fitness_function, true_fitness_params)

    return all_generations, best_solutions, best_fitnesses, true_fitnesses

def record_population_state(data, toolbox, true_fitness_function, true_fitness_params):
    # Unpack the data list
    population, all_generations, best_solutions, best_fitnesses, true_fitnesses = data

    # Record the current population, the best solution, and the true fitness if applicable
    all_generations.append([ind[:] for ind in population])
    best_individual = max(population, key=lambda ind: ind.fitness.values)  # Find the individual with the best fitness
    best_solutions.append(toolbox.clone(best_individual))  # Clone the best individual for recording
    best_fitnesses.append(best_individual.fitness.values[0])  # Record the best fitness value
    
    # If a true fitness function is provided, calculate the true fitness of the best solution
    if true_fitness_function is not None:
        if true_fitness_params is None:
            true_fitness_params = {}
        true_fitness = true_fitness_function(best_individual, **true_fitness_params)
        true_fitnesses.append(true_fitness[0])
    else:
        true_fitnesses.append(best_individual.fitness.values[0])


### Data generation

In [21]:
problem_name = 'f2_l-d_kp_20_878'
n_items, capacity, optimal, values, weights, items_dict = load_problem_KP(problem_name)

problem_info = {
    'number of items': n_items,
    'capacity': capacity,
    'optimal fitness': optimal,
    'values': values,
    'Weights': weights
}

number of items: 20
max weight: 878
values: [44 46 90 72 91 40 75 35  8 54 78 40 77 15 61 17 75 29 75 63]
weights: [92  4 43 83 84 68 92 82  6 44 32 18 56 83 25 96 70 48 14 58]
optimal solution: 1024


In [ ]:
fitness_params = {
    'items_dict': items_dict,
    'capacity': capacity,
    'penalty': 1
}

EA_params = {
    'NGEN': 100, # Number of generations
    'popsize': 100, # Population size
    'tournsize': 30, # Tournament selection size
    'MUTPB': 1, # Mutation probability
    'indpb': 0.2, # per-gene mutation probability
    'len_sol': n_items, # solution length
    'fitness_function': eval_ind_kp, # algorithm objective function
    'fitness_params': {'items_dict': items_dict, 'capacity': capacity, 'penalty': 1}, # objective function parameters
    'starting_solution': generate_zero_solution(n_items), # Specified starting solution for all individuals
    'true_fitness_function': None, # noise-less fitness function for performance evaluation
    'true_fitness_params': None, # noise-less fitnes function parameters
    'n_elite': 3
}

all_generations, best_solutions, best_fitnesses, true_fitnesses = EA(**EA_params)

In [7]:
print(best_solutions)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1], [0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 

In [8]:
print(best_fitnesses)

[0.0, 473.0, 736.0, 865.0, 959.0, 959.0, 990.0, 990.0, 1012.0, 1012.0, 1012.0, 1012.0, 1012.0, 1012.0, 1013.0, 1013.0, 1013.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0, 1024.0]


In [9]:
def extract_trajectory_data(best_solutions, best_fitnesses):
    # Extract unique solutions and their corresponding fitness values
    unique_solutions = []
    unique_fitnesses = []
    solution_iterations = []
    seen_solutions = {}

    for solution, fitness in zip(best_solutions, best_fitnesses):
        # Convert solution to a tuple to make it hashable
        solution_tuple = tuple(solution)
        if solution_tuple not in seen_solutions:
            seen_solutions[solution_tuple] = 1
            unique_solutions.append(solution)
            unique_fitnesses.append(fitness)
        else:
            seen_solutions[solution_tuple] += 1

    # Create a list of iteration counts for each unique solution
    for solution in unique_solutions:
        solution_tuple = tuple(solution)
        solution_iterations.append(seen_solutions[solution_tuple])

    return unique_solutions, unique_fitnesses, solution_iterations

In [10]:
def extract_transitions(unique_solutions):
    # Extract transitions between solutions over generations
    transitions = []

    for i in range(1, len(unique_solutions)):
        prev_solution = tuple(unique_solutions[i - 1])
        current_solution = tuple(unique_solutions[i])
        transitions.append((prev_solution, current_solution))

    return transitions

In [11]:
# def run_multiple_ea_runs(num_runs, NGEN, popsize, tournsize, MUTPB, indpb, starting_solution=None):
#     all_run_trajectories = []
#     for run in range(num_runs):
#         _, best_solutions, best_fitnesses = MutationOnlyEA(NGEN, popsize, tournsize, MUTPB, indpb, starting_solution)
#         unique_solutions, unique_fitnesses, solution_iterations = extract_trajectory_data(best_solutions, best_fitnesses)
#         transitions = extract_transitions(unique_solutions)
#         all_run_trajectories.append((unique_solutions, unique_fitnesses, solution_iterations, transitions))
#     return all_run_trajectories

In [12]:
def conduct_runs(num_runs, algorithm_function, param_dict):
    " Function conducts multiple algorithm runs "
    all_run_trajectories = []
    
    for run in range(num_runs):
        # Run the given algorithm with the provided parameters
        all_generations, best_solutions, best_fitnesses, true_fitnesses = algorithm_function(**param_dict)
        
        # Extract trajectories and transitions
        unique_solutions, unique_fitnesses, solution_iterations = extract_trajectory_data(best_solutions, best_fitnesses)
        transitions = extract_transitions(unique_solutions)
        
        # Store the results for each run
        all_run_trajectories.append((unique_solutions, unique_fitnesses, solution_iterations, transitions))
    
    return all_run_trajectories

In [13]:
data = conduct_runs(5, EA, EA_params)

In [14]:
print(data)

[([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1]], [0.0, 458.0, 821.0, 851.0, 903.0, 949.0, 958.0, 970.0, 1010.0, 1018.0, 1024.0], [1, 1, 1, 1, 1, 1, 1, 3, 4, 47, 40], [((0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0)), ((1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0), (1, 1, 

In [24]:
def save_data(data, problem_name, algo_name):
    folder_path = 'data/' +  problem_name
    file_name = algo_name + '.pkl'

    # Create the folder if it does not exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save the file within the folder
    file_path = os.path.join(folder_path, file_name)

    # Write file
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def save_parameters(parameters, problem_name, algo_name):
    folder_path = 'data/' +  problem_name
    file_name = algo_name + '.txt'

    # Create the folder if it does not exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save the file within the folder
    file_path = os.path.join(folder_path, file_name)

    # Write file
    with open(file_path, 'w') as file:
        json.dump(str(parameters), file)
    
def save_problem(problem_info, problem_name):
    folder_path = 'data/' +  problem_name
    file_name = 'info' + '.txt'

    # Create the folder if it does not exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # Create a serializable version of dictionary
    serializable_info = {
        key: value if isinstance(value, (int, float, str, bool, list, dict)) else str(value)
        for key, value in problem_info.items()
    }

    # Save the file within the folder
    file_path = os.path.join(folder_path, file_name)

    # Write file
    with open(file_path, 'w') as file:
        json.dump(serializable_info, file)

In [25]:
problem_name = 'f2_l-d_kp_20_878'
algo_name = 'EA_g100_p100_V2'

save_data(data, problem_name, algo_name)
save_parameters(EA_params, problem_name, algo_name)
save_problem(problem_info, problem_name)